In [71]:
#simple linear regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql
import time

%matplotlib qt

def load_dbscore_data():
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment2')
    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "select * from db_score"
    curs.execute(sql)
    
    data  = curs.fetchall()
    
    curs.close()
    conn.close()
    
    #X = [ (t['attendance'], t['homework'], t['midterm'] ) for t in data ]
    X = [ ( t['midterm'] ) for t in data ]
    X = np.array(X)
    
    y = [ (t['score']) for t in data]
    y = np.array(y)

    return X, y

X, y = load_dbscore_data()

# y = mx + c

from matplotlib import pyplot as plt
from celluloid import Camera
fig = plt.figure()
ax = fig.subplots()

camera = Camera(fig)
x = np.linspace(0, 2 * np.pi)

import statsmodels.api as sm
X_const = sm.add_constant(X)

model = sm.OLS(y, X_const)
ls = model.fit()

print(ls.summary())


ls_c = ls.params[0]
ls_m = ls.params[1]

c=ls_c
m=ls_m

y_pred = m*X + c

#plt.scatter(X, y) 
#plt.plot([min(X), max(X)], [min(y_pred), max(y_pred)], color='red')
#plt.show()

def gradient_descent_naive(X, y):

    epochs = 100000
    min_grad = 0.0001
    learning_rate = 0.001
    
    m = 0.0
    c = 0.0
    
    n = len(y)
    
    c_grad = 0.0
    m_grad = 0.0
    
    for epoch in range(epochs):
        
        for i in range(n):
            y_pred = m * X[i] + c
            m_grad += 2*(y_pred-y[i]) * X[i]
            c_grad += 2*(y_pred - y[i])

        c_grad /= n
        m_grad /= n
        
        m = m - learning_rate * m_grad
        c = c - learning_rate * c_grad
        
        y_pred /= n
        
        if ( epoch % 1000 == 0):
            print("epoch %d: m_grad=%f, c_grad=%f, m=%f, c=%f" %(epoch, m_grad, c_grad, m, c) ) 
            
            ax.scatter(X, y) 
            ax.plot([min(X), max(X)], [min(m*X+c), max(m*X+c)])
            plt.show()
            camera.snap()
        
        if ( abs(m_grad) < min_grad and abs(c_grad) < min_grad ):
            break
        
    return m, c

start_time = time.time()
m, c = gradient_descent_naive(X, y)
end_time = time.time()

print("%f seconds" %(end_time - start_time))

print("\n\nFinal:")
print("gdn_m=%f, gdn_c=%f" %(m, c) )
print("ls_m=%f, ls_c=%f" %(ls_m, ls_c) )

animation = camera.animate(interval=50, blit=True)
#animation.save('gradient descent.gif')

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     306.8
Date:                Mon, 26 Oct 2020   Prob (F-statistic):           9.65e-31
Time:                        20:53:58   Log-Likelihood:                -312.21
No. Observations:                  92   AIC:                             628.4
Df Residuals:                      90   BIC:                             633.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         24.3479      2.390     10.186      0.0

In [ ]:
#multiple linear regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql
import time

def load_dbscore_data():
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment2')
    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "select * from db_score"
    curs.execute(sql)
    
    data  = curs.fetchall()
    
    curs.close()
    conn.close()
    
    X = [ ( t['attendance'],t['homework'],t['midterm'] ) for t in data ]
    X = np.array(X)
    
    y = [ (t['score']) for t in data]
    y = np.array(y)

    return X, y

X, y = load_dbscore_data()
X1, X2, X3 = np.hsplit(X, 3) #3개의 배열로 분리

import statsmodels.api as sm #Least Square
X_const = sm.add_constant(X)

model = sm.OLS(y, X_const)
ls = model.fit()

print(ls.summary())

ls_c = ls.params[0]
ls_m1 = ls.params[1]
ls_m2 = ls.params[2]
ls_m3 = ls.params[3]

def gradient_descent_vectorized(X, y):
    #epochs = 100000
    epochs = 1000000
    min_grad = 0.0001
    #learning_rate = 0.001
    learning_rate = 0.00001
    
    m1 = 0.0
    m2 = 0.0
    m3 = 0.0
    c = 0.0
    
    n = len(y)
    
    c_grad = 0.0
    m1_grad = 0.0
    m2_grad = 0.0
    m3_grad = 0.0

    
    for epoch in range(epochs):    
       
        y_pred = (m1*X1) + (m2*X2) + (m3*X3) + c #X를 통으로 집어넣는다
        m1_grad = (2*(y_pred - y)*X1).sum()/n # 한번에 계산
        m2_grad = (2*(y_pred - y)*X2).sum()/n # 한번에 계산
        m3_grad = (2*(y_pred - y)*X3).sum()/n # 한번에 계산
        c_grad = (2*(y_pred - y)).sum()/n
        m1 = m1 - learning_rate * m1_grad
        m2 = m2 - learning_rate * m2_grad
        m3 = m3 - learning_rate * m3_grad
        c = c - learning_rate * c_grad    
        
        if ( epoch % 1000 == 0):
            print("epoch %d: m1_grad=%f, m2_grad=%f, m3_grad=%f, c_grad=%f " %(epoch, m1_grad, m2_grad, m3_grad, c_grad) )
            print("m1=%f, m2=%f, m3=%f, c=%f \n" %(m1,m2,m3,c))
    
        if ( abs(m1_grad) < min_grad and abs(m2_grad) < min_grad and abs(m3_grad) < min_grad and  abs(c_grad) < min_grad ):
            break

    return m1,m2,m3,c

start_time = time.time()
m1,m2,m3, c = gradient_descent_vectorized(X, y)
end_time = time.time()

print("%f seconds" %(end_time - start_time))

print("\n\nFinal:")
print("gdv_m1=%f, gdv_m2=%f, gdv_m3=%f, gdv_c=%f" %(m1,m2,m3,c) )
print("ls_m1=%f, ls_m2=%f, ls_m3=%f, ls_c=%f" %(ls_m1,ls_m2,ls_m3, ls_c) )
